# 코드트리 채점기

소요시간: NaN

작성일시: 24.04.08 21:03:31

출처: https://www.codetree.ai/training-field/frequent-problems/problems/codetree-judger/description?page=1&pageSize=20

### 접근 방법
1st
- 변수
  - 대기 큐 (heapq)
  - 대기 url 세트
  - 휴식 큐 (heapq)
  - 채점기 별 진행 info 리스트
  - 진행 도메인 세트
  - 종료 히스토리 해시 (도메인: (start, gap))
- 접근
  - 대기 큐에서 작업을 선택할 때 우선도와 대기 시작 시간이 짧은 순서대로 꺼내야 하므로 min heap 우선순위 큐를 사용.
  - 가용한 채점기를 선택할 때도 우선순위 큐를 사용.
  - 대기중인 작업의 url, 채점 진행중인 작업의 도메인 중복 여부 체크를 위해 set을 사용.
  - 종료된 작업과의 시간 차이를 확인하기 위해 dict를 사용.
- 시간 초과...

2nd
- 개선 사항
  - 대기 큐에서 가능한 작업을 선택할 때, 작업의 도메인이 채점중인 다른 작업의 도메인과 일치하면 안되고, 동일한 도메인의 채점 기록에서 현재 시각이 기록의 start + 3 * gap보다 크거나 같아야 하는 조건이 있음.
  - 우선순위 큐에서 꺼낸 작업이 위의 조건 때문에 선택받지 못하고 후순위로 밀리는 비효율 발생.
  - 대기 큐를 도메인별로 분리 저장하고, 도메인 별로 우선순위 큐에서 작업을 하나씩만 꺼내서 조회하고, 이를 비교함으로써 비효율을 개선할 수 있음
- 성공 (5000ms -> 1500ms)

### 시간 복잡도 분석
O(QlogN)

### 새로 알게 된 것
.

### 주의할 점
.

### 기타 코멘트
.


In [ ]:
import heapq

Q = int(input())
wait_task_q = []
wait_url_set = set()
idle_judger_q = []
exec_info = []
exec_domain_set = set()
history = dict()

def get_domain(u):
    return u.split('/')[0]

def request_judge(t, p, u):
    if u not in wait_url_set:
        heapq.heappush(wait_task_q, (p, t, u))
        wait_url_set.add(u)

def prepare_judger(n, u):
    global idle_judger_q, exec_info
    idle_judger_q = list(range(1, n + 1))
    heapq.heapify(idle_judger_q)
    exec_info = [None] * (n + 1)
    request_judge(0, 1, u)

def stop_judge(t, j_id):
    if exec_info[j_id] is not None:
        s, u = exec_info[j_id]
        domain = get_domain(u)
        exec_info[j_id] = None
        exec_domain_set.remove(domain)
        heapq.heappush(idle_judger_q, j_id)
        history[domain] = (s, t-s)

def get_wait_q_size():
    return len(wait_task_q)

def start_judge(cur_time):
    if wait_task_q and idle_judger_q:
        judge_exceptions = []
        while wait_task_q:
            p, t, u = heapq.heappop(wait_task_q)
            domain = get_domain(u)
            if domain in exec_domain_set:
                judge_exceptions.append((p, t, u))
                continue
            if domain in history:
                s, g = history[domain]
                if cur_time < s + 3 * g:
                    judge_exceptions.append((p, t, u))
                    continue
            j_id = heapq.heappop(idle_judger_q)
            exec_info[j_id] = (cur_time, u)
            exec_domain_set.add(domain)
            wait_url_set.remove(u)
            break
        for ptu in judge_exceptions:
            heapq.heappush(wait_task_q, ptu)

for _ in range(Q):
    input_list = list(input().split())
    op_num = input_list[0]
    if op_num == '100':
        n, u = input_list[1:]
        prepare_judger(int(n), u)
    elif op_num == '200':
        t, p, u = input_list[1:]
        request_judge(int(t), int(p), u)
    elif op_num == '300':
        t = input_list[1]
        start_judge(int(t))
    elif op_num == '400':
        t, j_id = input_list[1:]
        stop_judge(int(t), int(j_id))
    else:
        print(get_wait_q_size())

In [ ]:
import heapq
from collections import defaultdict

Q = int(input())
wait_task_q_by_domain_hash = defaultdict(list)
wait_url_set = set()
idle_judger_q = []
exec_info = []
exec_domain_set = set()
history = dict()

def get_domain(u):
    return u.split('/')[0]

def request_judge(t, p, u):
    if u not in wait_url_set:
        domain = get_domain(u)
        heapq.heappush(wait_task_q_by_domain_hash[domain], (p, t, u))
        wait_url_set.add(u)

def prepare_judger(n, u):
    global idle_judger_q, exec_info
    idle_judger_q = list(range(1, n + 1))
    heapq.heapify(idle_judger_q)
    exec_info = [None] * (n + 1)
    request_judge(0, 1, u)

def stop_judge(t, j_id):
    if exec_info[j_id] is not None:
        s, u = exec_info[j_id]
        domain = get_domain(u)
        exec_info[j_id] = None
        exec_domain_set.remove(domain)
        heapq.heappush(idle_judger_q, j_id)
        history[domain] = (s, t-s)

def get_wait_q_size():
    return sum(map(lambda x: len(wait_task_q_by_domain_hash[x]), wait_task_q_by_domain_hash))

def start_judge(cur_time):
    if idle_judger_q:
        target_domain = None
        target_task = None
        for d in wait_task_q_by_domain_hash:
            if d in exec_domain_set:
                continue
            if d in history:
                s, g = history[d]
                if cur_time < s + 3 * g:
                    continue
            wait_q = wait_task_q_by_domain_hash[d]
            p, t, u = wait_q[0]
            if target_domain is None or (p, t, u) < target_task:
                target_domain = d
                target_task = (p, t, u)
        if target_domain is not None:
            p, t, u = target_task
            j_id = heapq.heappop(idle_judger_q)
            exec_info[j_id] = (cur_time, u)
            exec_domain_set.add(target_domain)
            wait_url_set.remove(u)
            heapq.heappop(wait_task_q_by_domain_hash[target_domain])
            if not wait_task_q_by_domain_hash[target_domain]:
                wait_task_q_by_domain_hash.pop(target_domain)

for _ in range(Q):
    input_list = list(input().split())
    op_num = input_list[0]
    if op_num == '100':
        n, u = input_list[1:]
        prepare_judger(int(n), u)
    elif op_num == '200':
        t, p, u = input_list[1:]
        request_judge(int(t), int(p), u)
    elif op_num == '300':
        t = input_list[1]
        start_judge(int(t))
    elif op_num == '400':
        t, j_id = input_list[1:]
        stop_judge(int(t), int(j_id))
    else:
        print(get_wait_q_size())
